In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("RegressaoLinear").getOrCreate()

23/12/21 10:39:08 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/21 10:39:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 10:39:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
carros = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/Carros.csv", header=True, inferSchema=True, sep=";")
print(carros.count())
carros.show(5)

32
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [8]:
# Usando RFormula para separar o dataframe em conjunto de variaveis dependente e independentes
from pyspark.ml.feature import RFormula
Rformula = RFormula(formula="HP ~ Consumo + Cilindros + Cilindradas", featuresCol="independent_features", labelCol="dependent_variable")
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select("independent_features", "dependent_variable").show(truncate=False)

+--------------------+------------------+
|independent_features|dependent_variable|
+--------------------+------------------+
|[21.0,6.0,160.0]    |110.0             |
|[21.0,6.0,160.0]    |110.0             |
|[228.0,4.0,108.0]   |93.0              |
|[214.0,6.0,258.0]   |110.0             |
|[187.0,8.0,360.0]   |175.0             |
|[181.0,6.0,225.0]   |105.0             |
|[143.0,8.0,360.0]   |245.0             |
|[244.0,4.0,1467.0]  |62.0              |
|[228.0,4.0,1408.0]  |95.0              |
|[192.0,6.0,1676.0]  |123.0             |
|[178.0,6.0,1676.0]  |123.0             |
|[164.0,8.0,2758.0]  |180.0             |
|[173.0,8.0,2758.0]  |180.0             |
|[152.0,8.0,2758.0]  |180.0             |
|[104.0,8.0,472.0]   |205.0             |
|[104.0,8.0,460.0]   |215.0             |
|[147.0,8.0,440.0]   |230.0             |
|[324.0,4.0,787.0]   |66.0              |
|[304.0,4.0,757.0]   |52.0              |
|[339.0,4.0,711.0]   |65.0              |
+--------------------+------------

In [9]:
# Separando dados em Treino e Teste
CarrosTreino, CarrosTeste = carrosrf.randomSplit([0.8, 0.2])
print(CarrosTreino.count())
print(CarrosTeste.count())

26
6


In [10]:
# Criando modelo de Regressão Linear
from pyspark.ml.regression import LinearRegression
reglin = LinearRegression(featuresCol="independent_features", labelCol="dependent_variable", maxIter=1000,
                          loss="squaredError", standardization=True)
modelo = reglin.fit(CarrosTreino)

23/12/21 10:54:14 WARN Instrumentation: [3777ccca] regParam is zero, which might cause numerical instability and overfitting.


In [11]:
previsao = modelo.transform(CarrosTeste)
previsao.select("dependent_variable", "prediction").show()

+------------------+------------------+
|dependent_variable|        prediction|
+------------------+------------------+
|             215.0|223.40867009469747|
|             150.0|222.40209233177725|
|             150.0|222.00080642027706|
|             180.0|186.63177927273387|
|             105.0|155.58122394294458|
|              66.0| 85.47664839263673|
+------------------+------------------+



In [12]:
# Avaliando performance do modelo e medindo o erro
from pyspark.ml.evaluation import RegressionEvaluator

avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="dependent_variable", metricName="rmse")
rmse = avaliar.evaluate(previsao)
print(rmse)

47.39666896954159
